# Sobre linearidade

Os problemas que temos visto de Programação Inteira tem por objectivo calcular uma solução (ou a melhor solução, no caso de ser um problema de optimização) para um problema modelado como um conjunto de relações *lineares*, i.e., onde só são permitidas multiplicações em que um dos factores é constante.
O relaxamento desta condição torna o problema muito mais difícil, e muitos solvers não lhe conseguem dar resposta, ou apenas conseguem responder em situações muito limitadas.

### Exercício 1

Considere o exemplo de utilização do SCIP para encontrar $x$ e $y$ que minimizem $3x+4y$ satisfazendo as seguintes restrições:

$$
\left\{
\begin{array}{l}
5x + 6y \leq 200\\
2x + 80y \ge 150\\
0 \leq x \leq 100\\
0 \leq y \leq 100
\end{array}
\right.
$$

In [1]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')

x = solver.IntVar(0.0,100,"x")
y = solver.IntVar(0.0,100,"y")

solver.Add(5*x + 6*y <= 200)
solver.Add(2*x + 80*y >= 150)

solver.Minimize(3*x + 4*y)

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print ("x = ",round(x.solution_value()))
    print ("y = ",round(y.solution_value()))
    print ("objectivo = ",round(solver.Objective().Value()))
else:
    print('No solution found.')

x =  0
y =  2
objectivo =  8


Altere o programa de forma a acrescentar a restrição (não linear)  $ x y \ge 1$ e veja como ele se comporta.

In [2]:
solver.Add(x*y>=1)

TypeError: 

Como se deve ter apercebido, o *wrapper* do SCIP assinala logo um erro de tipo, não perminido assim o produto de duas variáveis. 
O OR-Tools disponibiliza outras ferramentas de programação inteira que permitem este tipo de restrições, como é o caso do [CP-SAT solver](https://developers.google.com/optimization/cp/cp_solver).

## CP-SAT solver

O CP-SAT solver é uma ferramenta de programação inteira do OR-Tools que consegue lidar com a multiplicação de variáveis, embora de uma forma especial. 

A forma de interagir com o CP-SAT difere em alguns detalhes da do SCIP. Para ilustrar isso, apresentamos a seguir  a resolução comentada do exercício 1 em CP-SAT. Para executar este código é necessária a biblioteca pandas, que pode instalar utilizando o comando `pip install pandas`. 

In [3]:
# Importar biblioteca
from ortools.sat.python import cp_model

# Cria o modelo CP-SAT
model = cp_model.CpModel()

# Cria as variáveis
x = model.NewIntVar(0, 100, 'x')
y = model.NewIntVar(0, 100, 'y')

# Adiciona as restrições ao modelo
model.Add(5*x + 6*y <= 200)
model.Add(2*x + 80*y >= 150)

# Define o objectivo
model.Minimize(3*x + 4*y)

# Cria um solver CP-SAT a solver and solves the model.
solver = cp_model.CpSolver()

# Invoca o solver com o modelo criado
status = solver.Solve(model)

# Interpreta os resultados
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('x = %i' % solver.Value(x))
    print('y = %i' % solver.Value(y))
    print ("objectivo = ", int(solver.ObjectiveValue()))
else:
    print('No solution found.')

x = 0
y = 2
objectivo =  8


Se tentar acrescentar a restrição $x y\ge 1$ através de `model.Add(x*y <= 1)` verá que obtém também um erro de tipo.
No CP-SAT a multiplicação de variáveis tem um tratamento especial.

A multiplicação de variáveis no CP-SAT não é uma operação como a multiplicação de um escalar por uma variável. Para cada multiplicação de variáveis é necessário criar uma variável adicional que representa essa multiplicação. É depois esta nova variável que entra nas restrições que são acrescentadas.

In [4]:
# Importar biblioteca
from ortools.sat.python import cp_model

# Cria o modelo CP-SAT
model = cp_model.CpModel()

# Cria as variáveis
x = model.NewIntVar(0, 100, 'x')
y = model.NewIntVar(0, 100, 'y')

# Adiciona as restrições ao modelo
model.Add(5*x + 6*y <= 200)
model.Add(2*x + 80*y >= 150)

# Cria uma variável adicional que representa a multiplicação das variáveis
xy = model.NewIntVar(0, 100*100, "xy")
model.AddMultiplicationEquality(xy, [x,y])  # xy = x*y
# Acescenta a restrição x*y>=1
model.Add(xy >= 1)

# Define o objectivo
model.Minimize(3*x + 4*y)

# Cria um solver CP-SAT a solver and solves the model.
solver = cp_model.CpSolver()

# Invoca o solver com o modelo criado
status = solver.Solve(model)

# Interpreta os resultados
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('x = %i' % solver.Value(x))
    print('y = %i' % solver.Value(y))
    print ("objectivo = ", int(solver.ObjectiveValue()))
else:
    print('No solution found.')

x = 1
y = 2
objectivo =  11


Tem [aqui](https://github.com/d-krupke/cpsat-primer) um breve tutorial do CP-SAT.

# Mais um problema de optimização linear

Um hospital trabalha todos os dias da semana e tem que ter um número mínimo de enfermeiros para assegurar o seu bom funcionamento. Os enfermeiros trabalham em turnos, e há 3 turnos: manhã, tarde ou noite.
As regras de funcionamento do hospital, quanto ao corpo de enfermagem, são as seguintes:

- Todos os turnos têm pelo menos N enfermeiros.
- Os turnos da manhã, por serem mais movimentados, têm no mínimo, mais 30% de enfermeiros do que é exigido nos restantes turnos.
- Um enfermeiro não pode trabalhar dois turnos seguidos.
- Todos os enfermeiros têm que ter, pelo menos, um dia de folga.

Dado o número N de enfermeiros que se quer assegurar por turno, pretende-se saber qual será o números mínimo de enfermeiros que o hospital deve contratar.

### Exercício 2

Faça a análise do problema acima apresentado. Primeiro, indique claramente as variáveis que vai necessitar e a sua interpretação. Depois, escreva formalmente as restrições que modelam o problema. Por fim, defina o objectivo do problema. 

(completar)

### Exercício 3

Codifique em Python uma função `nurses(N)` que calcula o número mínimo de enfermeiros que um hospital deve contratar.

In [1]:
# completar

from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')

N = 2
Nmax = 3*N
T = 3*7

# Variables x[n, t]: whether nurse n works shift t.
x = {}
for n in range(Nmax):
    for t in range(T):
        x[n,t] = solver.BoolVar(f"x[{n},{t}]")

# All shifts have at least N nurses.
for t in range(T):
    if (t%3) != 0:
        solver.Add(solver.Sum([x[n,t] for n in range(Nmax)]) >= N)
    else:
        # Morning shifts have a 30% higher minimum.
        solver.Add(solver.Sum([x[n,t] for n in range(Nmax)]) >= 1.3*N)

# No consecutive shifts.
for n in range(Nmax):
    for t in range(T-1):
        solver.Add(x[n,t] + x[n,t+1] <= 1)
    
    # Last shift of the week is followed by the next shift of the following week.
    solver.Add(x[n, T-1] + x[n, 0] <= 1)

D = 7
# Variables f[n,d]: whether nurse N works day d.
f = {}
for n in range(Nmax):
    for d in range(D):
        f[n, d] = solver.BoolVar(f"f[{n},{d}]")

for n in range(Nmax):
    # f[n,d] is 1 only if nurse n works on weekday d.
    for d in range(D):
        for t in range(3):
            solver.Add(f[n, d] >= x[n, 3*d + t])
        
    # Each nurse has at least one day off.
    solver.Add(solver.Sum([f[n, d] for d in range(D)]) <= 6)

# Variables w[n]: whether nurse N works any shift.
w = {}
for n in range(Nmax):
    w[n] = solver.BoolVar(f"w[{n}]")
    
# w[n] is 0 only if nurse n doesn't work any shift.
for n in range(Nmax):
    for t in range(T):
        solver.Add(w[n] >= x[n,t])
       
solver.Minimize(solver.Sum(w[n] for n in range(Nmax)))

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    for n, t in x.keys():
        print (f"x[{n},{t}] = ",round(x[n,t].solution_value()))
        
    print ("objectivo = ",round(solver.Objective().Value()))
else:
    print('No solution found.')

x[0,0] =  1
x[0,1] =  0
x[0,2] =  1
x[0,3] =  0
x[0,4] =  0
x[0,5] =  0
x[0,6] =  1
x[0,7] =  0
x[0,8] =  1
x[0,9] =  0
x[0,10] =  1
x[0,11] =  0
x[0,12] =  0
x[0,13] =  1
x[0,14] =  0
x[0,15] =  0
x[0,16] =  1
x[0,17] =  0
x[0,18] =  1
x[0,19] =  0
x[0,20] =  0
x[1,0] =  0
x[1,1] =  0
x[1,2] =  0
x[1,3] =  0
x[1,4] =  0
x[1,5] =  0
x[1,6] =  0
x[1,7] =  0
x[1,8] =  0
x[1,9] =  0
x[1,10] =  0
x[1,11] =  0
x[1,12] =  0
x[1,13] =  0
x[1,14] =  0
x[1,15] =  0
x[1,16] =  0
x[1,17] =  0
x[1,18] =  0
x[1,19] =  0
x[1,20] =  0
x[2,0] =  0
x[2,1] =  1
x[2,2] =  0
x[2,3] =  0
x[2,4] =  1
x[2,5] =  0
x[2,6] =  1
x[2,7] =  0
x[2,8] =  0
x[2,9] =  1
x[2,10] =  0
x[2,11] =  0
x[2,12] =  1
x[2,13] =  0
x[2,14] =  1
x[2,15] =  0
x[2,16] =  0
x[2,17] =  0
x[2,18] =  1
x[2,19] =  0
x[2,20] =  1
x[3,0] =  0
x[3,1] =  0
x[3,2] =  0
x[3,3] =  0
x[3,4] =  0
x[3,5] =  0
x[3,6] =  0
x[3,7] =  0
x[3,8] =  0
x[3,9] =  0
x[3,10] =  0
x[3,11] =  0
x[3,12] =  0
x[3,13] =  0
x[3,14] =  0
x[3,15] =  0
x[3,16] =  0
